In [2]:
from gurobipy import *
import numpy as np

## 1.1 introduction
a un ensemble d'alternatives evaluees selon n citeres

In [76]:
a=np.matrix('32 42 26;26 45 35;39 43 18;45 23 36;37 27 37')

In [77]:
a

matrix([[32, 42, 26],
        [26, 45, 35],
        [39, 43, 18],
        [45, 23, 36],
        [37, 27, 37]])

In [35]:
print(np.ndarray.max((a[1,:]-a[2,:])))

[[17]]


In [32]:
print(type([-13,2,17]))

<class 'list'>


In [27]:
b=np.ones((5,5))
print(b[1,1])

1.0


## 1.2 min-max regret

In [43]:
#PMR the pairwise max regret, le max regret entre deux alternative parmi toutes les citeres
def pmr(matrice):    
    nb_alt,nb_citere=matrice.shape
    matrice_pmr=np.zeros((nb_alt,nb_alt))
    for i in range(nb_alt):
        for j in range(nb_alt):
            if i!=j:               
                cpt=np.ndarray.max(np.array(matrice[i,:]-matrice[j,:]))
                matrice_pmr[i,j]=cpt
    return matrice_pmr

In [44]:
pmr_matrice=pmr(a)

In [45]:
pmr_matrice

array([[ 0.,  6.,  8., 19., 15.],
       [ 9.,  0., 17., 22., 18.],
       [ 7., 13.,  0., 20., 16.],
       [13., 19., 18.,  0.,  8.],
       [11., 11., 19.,  4.,  0.]])

In [48]:
print(np.ndarray.max(pmr_matrice))
print(pmr_matrice.shape)

22.0
(5, 5)


In [46]:
#PR : un des alternatives qui a le max PMR pour une solution donnees

In [54]:
def max_regret(pmrM):
    nb_alt=pmrM.shape[0]
    max_regret=np.zeros(nb_alt)
    for i in range(nb_alt):
        max_regret[i]=np.ndarray.max(pmrM[i,:])
    return max_regret

In [55]:
max_regret=max_regret(pmr_matrice)
print(max_regret)

[19. 22. 20. 19. 19.]


In [81]:
W=np.array([0.326851,0.318176,0.354973])
def preference(sol1,sol2):
    if(np.sum(W*sol1)<np.sum(W*sol2)):
        return True
    else:
        return False

In [83]:

sol1=np.ndarray.tolist(a[0,:])
sol2=np.ndarray.tolist(a[3,:])
print(preference(sol1,sol2))

[[32 42 26]
 [26 45 35]
 [39 43 18]
 [45 23 36]
 [37 27 37]]
True


In [106]:
#mettre les donnees dans un fichier et les lire
#une valeur pour nb des solution et pour les criteres
def minmax(pmrM,max_regret,a):
    m = Model("s0")
    #alpha = m.addVar(vtype=GRB.CONTINUOUS, name="alpha") 
    #m.setObjective(alpha,GRB.MAXIMIZE)
    
    #dans une liste
    w1=m.addVar(vtype=GRB.CONTINUOUS, name="w1")
    w2=m.addVar(vtype=GRB.CONTINUOUS, name="w2")
    w3=m.addVar(vtype=GRB.CONTINUOUS, name="w3")
    
    index=np.argmin(max_regret)
    print("la solution qui a min regret est solution ",index+1)
    sonpire=np.ndarray.argmax(pmrM[index,:])
    print("son pire adversaire est solution ",sonpire+1)
    sol1=np.ndarray.tolist(a[index,:])
    sol2=np.ndarray.tolist(a[sonpire,:])
    if(preference(sol1,sol2)):
        coeff=np.array(sol1)-np.array(sol2)
        m.setObjective(-5*w1+15*w2-11*w3,GRB.MAXIMIZE)
        m.addConstr(w1+w2+w3==1)
        m.addConstr(32*w1+42*w2+26*w3<=45*w1+23*w2+36*w3)
        m.optimize()
        print(m.objVal)

In [107]:
minmax(pmr_matrice,max_regret,a)

la solution qui a min regret est solution  1
son pire adversaire est solution  4
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x9652b34f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.09s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7500000e+30   1.093750e+30   3.750000e+00      0s
       2    3.1250000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.12 seconds
Optimal objective  3.125000000e+00
3.125
